# GPU 확인

In [1]:
!nvidia-smi

Wed Aug 25 14:02:41 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    35W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Library

In [2]:
import os
import pandas as pd
from PIL import Image
import tqdm
import glob

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from torchvision import models

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import torch.optim as optim
from sklearn.model_selection import KFold

# 변수설정

In [3]:
train_dir = '/opt/ml/input/data/train'
train_images_dir = train_dir + '/images/'
train_label = train_dir + '/train.csv'
test_dir = '/opt/ml/input/data/eval'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# TrainDataList 만들기 <- 미완성 DataSet class에서 마스크 유무에 따른 라벨을 추가해줘야함

In [67]:
pd_train = pd.read_csv(train_label)
train_path_list = [ train_images_dir+'/'+x +'/' for x in pd_train['path']]
def create_train_data_list(path_list):
    count = 0
    train_data_list = []
    train_label_list = []
    train_mask_label_list = []
    for data_path in path_list:
        train_data_list.append(glob.glob(data_path+'*'))
        temp_mask_label = [mask_data for mask_data in os.listdir(data_path) if not '_' in mask_data or ('incorrect' in mask_data and not '_incorrect' in mask_data )]
        train_mask_label_list = [0 if 'incorrect_mask' in mask_data else 1 if 'mask' in mask_data else 2 for mask_data in temp_mask_label]
#         print(temp_mask_label[0], train_mask_label_list[0])
        temp_train_label = data_path.split('_')
        train_label_list.append([temp_train_label[1],temp_train_label[3].rstrip('/')])
    print(temp_mask_label[0:7], train_mask_label_list[0:7])
    return [train_data_list, train_mask_label_list, train_label_list] #copy.deepcopy(train_data_list)

temp_train_info = create_train_data_list(train_path_list)
# print(train_info[1])

['mask2.jpg', 'mask4.jpg', 'mask5.jpg', 'mask3.jpg', 'incorrect_mask.jpg', 'mask1.jpg', 'normal.jpg'] [1, 1, 1, 1, 0, 1, 2]


# Label 만들어 주기

|class|mask|gender|age|
|:-:|:-:|:-:|:-:|
|0|1|0|0|
|1|1|0|1|
|2|1|0|2|
|3|1|1|0|
|4|1|1|1|
|5|1|1|2|
|6|0|0|0|
|7|0|0|1|
|8|0|0|2|
|9|0|1|0|
|10|0|1|1|
|11|0|1|2|
|12|2|0|0|
|13|2|0|1|
|14|2|0|2|
|15|2|1|0|
|16|2|1|1|
|17|2|1|2|

In [83]:
def check_mask(data):
    result = 1 if data == 1 else 0 if data == 0 else 2
    return result

def check_gender(gender):
    result = 1 if 'female' in gender else 0
    return result

def check_age(age):
    result_number = 0 if age < 30 else 2 if age >= 60 else 1
    return result_number

def label_classification(data, gender, age):
    result = 0
    # mask : wear
    if data == 1 and gender == 0 and age == 0:
        result = 0
    elif data == 1 and gender == 0 and age == 1:
        result = 1
    elif data == 1 and gender == 0 and age == 2:
        result = 2
    elif data == 1 and gender == 1 and age == 0:
        result = 3
    elif data == 1 and gender == 1 and age == 1:
        result = 4
    elif data == 1 and gender == 1 and age == 2:
        result = 5
    # mask : incorrect
    elif data == 0 and gender == 0 and age == 0:
        result = 6
    elif data == 0 and gender == 0 and age == 1:
        result = 7
    elif data == 0 and gender == 0 and age == 2:
        result = 8
    if data == 0 and gender == 1 and age == 0:
        result = 9
    elif data == 0 and gender == 1 and age == 1:
        result = 10
    elif data == 0 and gender == 1 and age == 2:
        result = 11
    # mask : not wear
    elif data == 2 and gender == 0 and age == 0:
        result = 12
    elif data == 2 and gender == 0 and age == 1:
        result = 13
    elif data == 2 and gender == 0 and age == 2:
        result = 14
    elif data == 2 and gender == 1 and age == 0:
        result = 15
    elif data == 2 and gender == 1 and age == 1:
        result = 16
    elif data == 2 and gender == 1 and age == 2:
        result = 17
    return result



data_path, mask_list, label_list = temp_train_info
# print(mask[0:7],label[0])
index = 0
new_label = []
count = 0
for label in label_list:
    gender, age = label
    gender_state = check_gender(gender)
    age_state = check_age(int(age))
    index *= count
    count += 1
    mask = mask_list[index:index+7]
    for target_mask in mask:
        mask_state = check_mask(target_mask)
        result_class = label_classification(mask_state, gender_state, age_state)
        # print(mask_state,gender_state,age_state,result_class)
        new_label.append(result_class)

new_data_path = [target_path for in_target_path in data_path for target_path in in_target_path ]
col_name = ['data_path', 'label']
new_info = []
series_data_path = pd.Series(new_data_path, name=col_name[0])
series_new_label = pd.Series(new_label, name=col_name[1])
df_new_info = pd.concat([series_data_path, series_new_label],axis=1)
df_new_info.to_csv('/opt/ml/new_label.csv',mode='w')

# series_new_label.head(14)


# 모델 정의

In [4]:
class MyModel(nn.Module):
    def __init__(self, num_classes: int = 1000):
        super(MyModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
#         print(1)
        x = self.features(x)
#         print(2)
        x = self.avgpool(x)
#         print(3)
        x = torch.flatten(x, 1)
#         print(4)
        x = self.classifier(x)
#         print(5)
        return x

    def init_param(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d): # init conv
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m,nn.BatchNorm2d): # init BN
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
            elif isinstance(m,nn.Linear): # lnit dense
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)

cnn_model = MyModel(num_classes=18).to(device)

In [61]:
class ConvolutionalNeuralNetworkClass(nn.Module):
    """
        Convolutional Neural Network (CNN) Class
    """
    def __init__(self,name='cnn',xdim=[1,28,28],
                 ksize=3,cdims=[32,64],hdims=[1024,128],ydim=10,
                 USE_BATCHNORM=False):
        super(ConvolutionalNeuralNetworkClass,self).__init__()
        self.name = name
        self.xdim = xdim
        self.ksize = ksize
        self.cdims = cdims
        self.hdims = hdims
        self.ydim = ydim
        self.USE_BATCHNORM = USE_BATCHNORM

        # Convolutional layers
        self.layers = []
        prev_cdim = self.xdim[0]
        for cdim in self.cdims: # for each hidden layer
            self.layers.append(
                nn.Conv2d(
                    # FILL IN HERE
                    in_channels = prev_cdim,
                    out_channels = cdim,
                    kernel_size = self.ksize,
                    stride=(1,1),
                    padding=self.ksize//2
                )) # convlution 
            if self.USE_BATCHNORM:
                self.layers.append(nn.BatchNorm2d(cdim)) # batch-norm
            self.layers.append(nn.ReLU(True))  # activation
            self.layers.append(nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))) # max-pooling 
            self.layers.append(nn.Dropout2d(p=0.5))  # dropout
            self.layers.append(nn.Softmax2d())
            prev_cdim = cdim

        # Dense layers
        self.layers.append(nn.Flatten())
        prev_hdim = prev_cdim*(self.xdim[1]//(2**len(self.cdims)))*(self.xdim[2]//(2**len(self.cdims)))
        for hdim in self.hdims:
            self.layers.append(nn.Linear(
                # FILL IN HERE
                prev_hdim, hdim, bias=True
                               ))
            self.layers.append(nn.ReLU(True))  # activation
            prev_hdim = hdim
        # Final layer (without activation)
        self.layers.append(nn.Linear(prev_hdim,self.ydim,bias=True))

        # Concatenate all layers 
        self.net = nn.Sequential()
        for l_idx,layer in enumerate(self.layers):
            layer_name = "%s_%02d"%(type(layer).__name__.lower(),l_idx)
            self.net.add_module(layer_name,layer)
        self.init_param() # initialize parameters
        
    def init_param(self):
        for m in self.modules():
            if isinstance(m,nn.Conv2d): # init conv
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m,nn.BatchNorm2d): # init BN
                nn.init.constant_(m.weight,1)
                nn.init.constant_(m.bias,0)
            elif isinstance(m,nn.Linear): # lnit dense
                nn.init.kaiming_normal_(m.weight)
                nn.init.zeros_(m.bias)
            
    def forward(self,x):
        return self.net(x)

C = ConvolutionalNeuralNetworkClass(
    name='cnn',xdim=[3,384,384],ksize=3,cdims=[64],
    hdims=[64],ydim=18).to(device)
loss = nn.CrossEntropyLoss()
optm = optim.Adam(C.parameters(),lr=1e-3)
C.init_param() # initialize parameters
C.train() # to train mode 
print ("Done.")

Done.


# Pretrained Model

## DenseNet161

In [4]:
class MyPreTrainedModel(nn.Module):
    def __init__(self):
        super(MyPreTrainedModel, self).__init__()
        self.dense161 = models.densenet161(pretrained=True)
        self.linear1 = nn.Linear(1000, 18)

    def forward(self, x):
        x = self.dense161(x)
        return self.linear1(x)

pretrained_model = MyPreTrainedModel().to(device)

## inception_V3

In [4]:
class MyPreTrainedModel1(nn.Module):
    def __init__(self):
        super(MyPreTrainedModel1, self).__init__()
        self.inception3 = models.inception_v3(pretrained=True)
        self.linear1 = nn.Linear(1000, 18)

    def forward(self, x):
        x = self.inception3(x)
        return self.linear1(x)

pretrained_model = MyPreTrainedModel1().to(device)

## VGG

In [4]:
class MyPreTrainedModel2(nn.Module):
    def __init__(self):
        super(MyPreTrainedModel2, self).__init__()
        self.vgg19 = models.vgg19(pretrained=True)
        self.linear1 = nn.Linear(1000, 18)

    def forward(self, x):
        x = self.vgg19(x)
        return self.linear1(x)

pretrained_model = MyPreTrainedModel2().to(device)

# PreTrained Summary

In [4]:
# print(pretrained_model)
summary(pretrained_model, input_size=(3,512,384))

NameError: name 'pretrained_model' is not defined

# DataSet 정의

In [5]:
data_info = pd.read_csv('/opt/ml/new_label.csv')
# data_info

In [6]:
class MaskDataSet(Dataset):
    def __init__(self, data_info, transform, train=True):
        self.row = 512
        self.column = 384
        self.x = data_info['data_path']
        self.y = data_info['label']
        self.transform = transform
    
    def __len__(self):
        len_dataset = len(self.x)
        return len_dataset
        
    def __getitem__(self, idx):
        image = Image.open(self.x[idx])
        x = self.transform(image)
        y = self.y[idx]

        return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=int)

    

In [7]:
train_iter = MaskDataSet(data_info = data_info, transform=transforms.Compose([transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    # transforms.CenterCrop(384),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])]),train=True)
len(train_iter)

18900

# collate 정의

In [8]:
def my_collate_fn(samples):
    collate_X = []
    collate_y = []
    ######################################TODO######################################
    collate_x = [sample['data_path'] for sample in samples]
    collate_y = [sample['label'] for sample in samples]
    ################################################################################
    return {'X': torch.stack(collate_X),
             'y': torch.stack(collate_y)}

# DataLoader 정의

In [9]:
BATCH_SIZE = 64
train_loader = torch.utils.data.DataLoader(train_iter,batch_size=BATCH_SIZE,shuffle=True)

In [10]:
# image, label = next(iter(train_loader))
# image, label

# Loss, Optimizer, epoch 정의

In [11]:
criterion = nn.CrossEntropyLoss()
# optm = optim.Adam(cnn_model.parameters(),lr=1e-3)
optm = optim.Adam(pretrained_model.parameters(),lr=1e-3)

In [12]:
NUM_EPOCH = 3

# GPU 확인

In [17]:
!nvidia-smi 


Wed Aug 25 13:58:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    37W / 250W |   3161MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 모델 Train 및 저장

## default

In [13]:
# torch.cuda.empty_cache()
# cnn_model.train()
# C.train()
pretrained_model.train()
# target = pretrained_model
for epoch in range(NUM_EPOCH):   # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for inputs, labels in tqdm.tqdm(train_loader):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs = inputs.to(device)
        labels = labels.to(device)
        # 변화도(Gradient) 매개변수를 0으로 만들고
        optm.zero_grad()

        # 순전파 + 역전파 + 최적화를 한 후
        logits = pretrained_model(inputs)

        _, preds = torch.max(logits, 1)
        loss = criterion(logits, labels)
        loss.backward() 
        optm.step() 
 

print('Finished Training')

  0%|          | 0/2363 [00:00<?, ?it/s]<ipython-input-6-d0d86192c460>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=int)
  0%|          | 0/2363 [00:00<?, ?it/s]


AttributeError: 'InceptionOutputs' object has no attribute 'dim'

## K-Fold

In [13]:
k_folds = 5
num_epochs = 3
loss_function = nn.CrossEntropyLoss()
model_name = 'VGG'
  
# For fold results
results = {}
  
# Set fixed random number seed
torch.manual_seed(42)
  
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)
    
# Start print
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_iter)):
    
    # Print
    print(f'FOLD {fold}')
    print('--------------------------------')
        
    # Sample elements randomly from a given list of ids, no replacement.
    test_loader = torch.utils.data.DataLoader(train_iter,batch_size=BATCH_SIZE,shuffle=True)

        
    # Run the training loop for defined number of epochs
    for epoch in range(0, num_epochs):

        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(train_loader, 0):
            
            # Get inputs
            inputs, targets = data
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Zero the gradients
            optm.zero_grad()

            # Perform forward pass
            outputs = pretrained_model(inputs)

            # Compute loss
            loss = criterion(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optm.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' %(i + 1, current_loss / 500))
                current_loss = 0.0
                
    # Process is complete.
    print('Training process has finished. Saving trained model.')

    # Print about testing
    print('Starting testing')
        
    # Saving the model
    save_path = f'./{model_name}-fold-{fold}.pth'
    torch.save(pretrained_model.state_dict(), save_path)
    if k_folds == 5:
        break

    # Evaluationfor this fold
#     correct, total = 0, 0
#     with torch.no_grad():

#         # Iterate over the test data and generate predictions
#         for i, data in enumerate(test_loader, 0):

#         # Get inputs
#         inputs, targets = data

#         # Generate outputs
#         outputs = pretrained_model(inputs)

#         # Set total and correct
#         _, predicted = torch.max(outputs.data, 1)
#         total += targets.size(0)
#         correct += (predicted == targets).sum().item()

#         # Print accuracy
#         print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
#         print('--------------------------------')
#         results[fold] = 100.0 * (correct / total)
    
# Print fold results
# print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
# print('--------------------------------')
# sum = 0.0
# for key, value in results.items():
#     print(f'Fold {key}: {value} %')
#     sum += value
# print(f'Average: {sum/len(results.items())} %')

--------------------------------
FOLD 0
--------------------------------
Starting epoch 1


<ipython-input-6-d0d86192c460>:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float), torch.tensor(y, dtype=int)


Starting epoch 2
Starting epoch 3
Training process has finished. Saving trained model.
Starting testing


# Test data 정의

In [14]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

# Inference

In [16]:
# meta 데이터와 이미지 경로를 불러옵니다.
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')
model_path = './denseNet_-fold-0.pth'

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])
dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
# pretrained_model = MyPreTrainedModel().to(device)
# pretrained_model.load_state_dict(torch.load(model_path))
pretrained_model.eval()
# model.eval()
# cnn_model.eval()
# C.eval()
# print(1)

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in tqdm.tqdm(loader):
    with torch.no_grad():
        # print(2)
        images = images.to(device)
        # pred = C(images)
        # print(3)
        pred = pretrained_model(images)
        #pred = cnn_model(images)
        # print(4)
        pred = pred.argmax(dim=-1)
        # print(pred)
        # print(5)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'submission.csv'), index=False)
print('test inference is done!')

100%|██████████| 12600/12600 [03:10<00:00, 66.28it/s]

test inference is done!


In [17]:
del pretrained_model